In [13]:
import fitz
import sys
import time
import fitz
import pprint

ERROR: Could not find a version that satisfies the requirement pprintp (from versions: none)
ERROR: No matching distribution found for pprintp


In [6]:
input_file = "/Users/sureshreddy/Downloads/diag_text.pdf"
output_file= "/Users/sureshreddy/Downloads/diag_text_layout.pdf"

print(fitz.__doc__)
t0 = time.time()
doc1 = fitz.open(input_file)
doc2 = fitz.open()
red = (1, 0, 0)
blue = (0, 0, 1)
green = (0, 1, 0)
gray = (0.9, 0.9, 0.9)

for page1 in doc1:
    blks = page1.get_text("blocks")  # read text blocks of input page
    # create new page in output with /MediaBox dimensions
    page2 = doc2.new_page(-1, width=page1.mediabox_size[0], height=page1.mediabox_size[1])
    # the text font we use
    page2.insert_font(fontfile=None, fontname="Helvetica")
    img = page2.new_shape()  # prepare /Contents object

    # calculate /CropBox & displacement
    disp = fitz.Rect(page1.cropbox_position, page1.cropbox_position)
    croprect = page1.rect + disp

    # draw original /CropBox rectangle
    img.draw_rect(croprect)
    img.finish(color=gray, fill=gray)

    for b in blks:  # loop through the blocks
        r = fitz.Rect(b[:4])  # block rectangle
        # add dislacement of original /CropBox
        r += disp
        img.draw_rect(r)  # surround block rectangle

        if b[-1] == 1:  # if image block ...
            color = red
            a = fitz.TEXT_ALIGN_CENTER
        else:  # if text block
            color = blue
            a = fitz.TEXT_ALIGN_LEFT

        img.finish(width=0.3, color=color)

        if r.is_empty:  # do not rely on meaningful rects
            print(
                "skipping text of empty rect at (%g, %g) on page %i"
                % (r.x0, r.y0, page1.number)
            )
        else:
            # insert text of the block using a small, indicative fontsize
            img.insert_textbox(
                r, b[4], fontname="/Helvetica", fontsize=8, color=color, align=a
            )

    img.commit()  # store /Contents of out page

# save output file
doc2.save(output_file, garbage=4, deflate=True, clean=True)
t1 = time.time()
print("total time: %g sec" % (t1 - t0))


PyMuPDF 1.22.5: Python bindings for the MuPDF 1.22.2 library.
Version date: 2023-06-21 00:00:01.
Built for Python 3.8 on darwin (64-bit).

total time: 0.00579596 sec


In [18]:
input_file = "/Users/sureshreddy/Downloads/diag_text.pdf"
output_file= "/Users/sureshreddy/Downloads/diag_text_draw.pdf"

doc = fitz.open(input_file)
page = doc[0]
paths = page.get_drawings()  # extract existing drawings

pp = pprint.PrettyPrinter(indent=4)


# pp.pprint(paths)

# this is a list of "paths", which can directly be drawn again using Shape
# -------------------------------------------------------------------------
#
# define some output page with the same dimensions
outpdf = fitz.open()
outpage = outpdf.new_page(width=page.rect.width, height=page.rect.height)
shape = outpage.new_shape()  # make a drawing canvas for the output page
# --------------------------------------
# loop through the paths and draw them
# --------------------------------------
for path in paths:
    # ------------------------------------
    # draw each entry of the 'items' list
    # ------------------------------------
    for item in path["items"]:  # these are the draw commands
        if item[0] == "l":  # line
            shape.draw_line(item[1], item[2])
        elif item[0] == "re":  # rectangle
            shape.draw_rect(item[1])
        elif item[0] == "qu":  # quad
            shape.draw_quad(item[1])
        elif item[0] == "c":  # curve
            shape.draw_bezier(item[1], item[2], item[3], item[4])
        else:
            raise ValueError("unhandled drawing", item)
    # ------------------------------------------------------
    # all items are drawn, now apply the common properties
    # to finish the path
    # ------------------------------------------------------
    shape.finish(
        fill=path["fill"],  # fill color
        color=path["color"],  # line color
        dashes=path["dashes"],  # line dashing
        even_odd=path.get("even_odd", True),  # control color of overlaps
        closePath=path["closePath"],  # whether to connect last and first point
        
        width=path["width"]   # line width
 
        )
# all paths processed - commit the shape to its page
shape.commit()
outpdf.save(output_file)